# Incident Response: OODA Loop - Kill Chain 3 Investigation walk-through
BTV Project Obsidian, 2022
<img align="right" width="100" height="100" src="https://cfc.blueteamvillage.org/media/call-for-content-2021/img/20200622_BTVillage_logos_RGB_pos_hcOC7Qx.png">

![BTV IR OODA loop](2022-BTV-IR-OODA.jpg)

## Agenda
* [Introduction](#Introduction)
* [OODA Loop](#OODA-Loop)
* [Key Questions](#Key-questions)
* [Early Alert](#Early-Alert)
* [Late Alert](#Late-Alert)
* [Lessons Learned](#Lessons-Learned)
* [Conclusion](#Conclusion)

## Introduction
* Triage, Scoping (see other walk-through)
  * Incident phases: Preparation > Identification > Containment > Eradication > Recovery > Lessons Learned
  * Document, document and document!
* OODA Loop
* Key questions

*presentation and notebook will be available after Defcon*

## OODA Loop
* Developed by US Colonel John Boyd from US Air Force in the sixties following his experience during Korean War
* Initially for dogfight but concept applicable to other domains, including security incidents
* Observe, Orient, Decide, Act
* Similar to Deming wheel in ISO standards (Plan Do Check Act) but at micro level

![OODA Loop](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/OODA.Boyd.svg/1920px-OODA.Boyd.svg.png)

## Key questions
Identify what you know and what you don't know - *[There are known knowns](https://en.wikipedia.org/wiki/There_are_known_knowns)*

* Is it a false-positive? known activity, testing (pentest or else)
* Do we have enough visibility to assess situation at a sufficient confidence level? Can evidence be preserved?
* Which users are impacted? which roles/permissions? Which systems are impacted?
* What stage of the attack it is? is it live/active?
* How did it start? Who/What is patient zero?
* Did lateral movement occurred?
* Which data attacker has accessed? - legal impact
* What is the business impact? services unavailability, information disclosure...

## Early Alert

User brent.socium got email notification that he logged in company jumphost but he didn't. He notified security.
What are you doing?

### ... > *Identification* > ...
Each question will result in one or multiple observations.
Those observations will orient investigation and result in decision and actions.

#### Start
* Open your ticket
* Acknowledge alert to user when manual report and with ticket reference
* Document what's known, what's unknown
* Regularly peer review

#### OODA 1
* Observe
  * Quick investigation of AD authentication logs shows that a password spray attack has happened and multiple accounts were compromised.
* Orient
  * Reach out to IT or helpdesk to validate no related activity, pentesting, testing or other
* Decide
  * Move to next phase, more identification, other
* Act

##### Msticpy base import
Set provider connection information in msticpyconfig.yaml

In [1]:
# Check we are running Python 3.6
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)

# Imports
import pandas as pd
import msticpy.nbtools as nbtools
from datetime import datetime,timedelta
import os
# path to config file
os.environ['MSTICPYCONFIG'] = '/home/user/QubesIncoming/lab1/msticpyconfig.yaml'

from msticpy.nbtools import *
from msticpy.data.data_providers import QueryProvider
from msticpy.common.wsconfig import WorkspaceConfig
from msticpy.nbtools.data_viewer import DataViewer
from msticpy.vis.matrix_plot import plot_matrix
from msticpy.nbtools import process_tree as ptree
print('Imports Complete')

/media/user/26fcf073-3d3f-450e-bd49-f3b505a46ab5/tmp/Lab1/others/env-notebooks/lib/python3.6/site-packages/msticpy/sectools/domain_utils.py:21: CryptographyDeprecationWarning: Python 3.6 is no longer supported by the Python core team. Therefore, support for it is deprecated in cryptography and will be removed in a future release.
  import cryptography as crypto


Imports Complete


In [2]:
# Interactive settings edit
# https://msticpy.readthedocs.io/en/latest/getting_started/SettingsEditor.html#using-mpconfigfile-to-check-and-manage-your-msticpyconfig-yaml
from msticpy.config import MpConfigFile, MpConfigEdit, MpConfigControls
mpconfig = MpConfigFile()
# mpconfig.load_default()
# mpconfig.view_settings()
mpconfig

In [3]:
# q_times = nbwidgets.QueryTime(units='hours', max_before=72, before=1, max_after=0)
q_times = nbwidgets.QueryTime(origin_time=datetime(2022, 2, 19), units='days', max_before=7, before=0, max_after=0)
q_times.display()

In [4]:
data_path = "/home/user/Lab1/others/obsidian-ir/draft"

##### Msticpy and Splunk

In [ ]:
%pip install msticpy[splunk]

In [ ]:
# Configuration
# if free splunk, 
#  * enable the 'allowRemoteLogin' setting in your server.conf file.
#  * need to patch splunk_driver.py and remove password from _SPLUNK_REQD_ARGS.
# you must keep username as 'admin' in msticpyconfig.yaml
splunk_prov = QueryProvider('Splunk')
splunk_prov.connect()

In [ ]:
# FIXME! HTTPError: HTTP 400 Bad Request -- Missing or malformed messages.conf stanza for SEARCHFACTORY:UNKNOWN_OP__index
# works in UI, maybe a free splunk limitation
splunk_query = r'''
index=wineventlogs (event.code=4624 OR event.code=4625) winlog.event_data.IpAddress!="::1" winlog.event_data.IpAddress!="-" winlog.event_data.TargetUserName!=*$
| rename winlog.event_data.IpAddress as SourceIP winlog.event_data.TargetUserName as UserAccount
| table _time index host.name event.code UserAccount SourceIP
| sort _time
'''
df_ad_auth = splunk_prov.exec_query(splunk_query)

In [ ]:
df_ad_auth_user_column = "UserAccount"
df_ad_auth_eventid_column = "event.code"
df_ad_auth_time_column = "_time"

##### Msticpy and Sentinel

In [ ]:
# Configuration
sentinel_prov = QueryProvider('AzureSentinel')
ws_config = WorkspaceConfig(workspace="Default")
sentinel_prov.connect(ws_config.code_connect_str)

In [ ]:
query_azure_ad_auth = f"""SigninLogs
| where TimeGenerated >= datetime({q_times.start})
| where TimeGenerated <= datetime({q_times.end})
| project TimeGenerated,Category,ResultType,ResultDescription,Identity,UserPrincipalName,Location,AutonomousSystemNumber,AppDisplayName,AuthenticationRequirement,IPAddress"""
df_azure_ad_auth = sentinel_prov.exec_query(query_azure_ad_auth)

In [ ]:
DataViewer(df_azure_ad_auth)

In [ ]:
df_azure_ad_auth.dtypes

In [ ]:
df_azure_ad_auth.mp_timeline.plot(
   title="Azure AD authentication result by User",
   # group_by="UserPrincipalName",
   # source_columns=["ResultDescription"],
   group_by="ResultDescription",
   source_columns=["UserPrincipalName"],
   time_column="TimeGenerated",
   legend="left",
   height=800
)

In [ ]:
plot_matrix(data=df_azure_ad_auth, x="ResultDescription", y="UserPrincipalName", title="Matrix Azure AD auth eventid vs user")

In [ ]:
query_ad_auth = f"""SecurityEvent
| where TimeGenerated >= datetime({q_times.start})
| where TimeGenerated <= datetime({q_times.end})
| where EventID == 4624 or EventID == 4625
| where not (TargetAccount endswith "$" or TargetUserName == "SYSTEM" or TargetUserName == "ANONYMOUS LOGON")
| project TimeGenerated,EventID,IpAddress,LogonTypeName,TargetAccount"""
df_ad_auth = sentinel_prov.exec_query(query_ad_auth)

In [ ]:
df_ad_auth_user_column = "TargetAccount"
df_ad_auth_eventid_column = "EventID"
df_ad_auth_time_column = "TimeGenerated"

##### Msticpy and Sumologic

In [ ]:
# Configuration
global_timezone = 'UTC'
sumo_prov = QueryProvider('Sumologic')
sumo_prov.connect()

In [ ]:
sumo_query = """_sourceCategory=BU/AZURE/AD
| json auto
| where ResultStatus != "Success"
| fields CreationTime,Operation,Workload,UserId,ClientIP,ResultStatus
| fields -_raw"""
df_azure_ad_auth2 = sumo_prov.exec_query(sumo_query, start_time=q_times.start, end_time=q_times.end, 
                 timeZone=global_timezone,
                 byReceiptTime=False)

In [ ]:
df_azure_ad_auth2.head(10)

In [ ]:
df_azure_ad_auth2.dtypes

In [ ]:
df_azure_ad_auth2['map.creationtime'] = pd.to_datetime(df_azure_ad_auth2['map.creationtime'])

In [ ]:
df_azure_ad_auth2.dtypes

In [ ]:
df_azure_ad_auth2.mp_timeline.plot(
   title="Azure AD authentication result by UserId",
   # group_by="map.userid",
   # source_columns=["map.resultstatus"],
   group_by="map.resultstatus",
   source_columns=["map.userid"],
   time_column="map.creationtime",
   legend="left",
   height=800
)

In [ ]:
query_ad_auth = """_sourceCategory=BU/DC/NXLOG_WINDOWS (eventid=4624 or eventid=4625)
| json auto
| where !(TargetUserName matches "*$" or TargetUserName matches "SYSTEM" or TargetUserName matches "ANONYMOUS LOGON")
| formatDate(_receipttime, "MM-dd-yyyy HH:mm:ss", "UTC") as receipttime
| fields receipttime,Category,eventid,TargetUserName,LogonType,IpAddress
| fields -_raw"""
df_ad_auth = sumo_prov.exec_query(query_ad_auth, start_time=q_times.start, end_time=q_times.end, 
                 timeZone=global_timezone,
                 byReceiptTime=False)

In [ ]:
# convert column to pandas datetime
df_ad_auth['map.receipttime'] = pd.to_datetime(df_ad_auth['map.receipttime'])

In [ ]:
df_ad_auth_user_column = "map.targetusername"
df_ad_auth_eventid_column = "map.eventid"
df_ad_auth_time_column = "map.receipttime"

##### Local files

In [ ]:
# not working, using pandas read_csv for now
qry_prov = QueryProvider("LocalData", data_paths=[data_path])

In [ ]:
print(qry_prov.list_queries())

In [5]:
df_ad_auth = pd.read_csv(os.path.join(data_path, 'kc3-wineventlogs-4624-4625.csv'))

In [6]:
df_ad_auth_user_column = "UserAccount"
df_ad_auth_eventid_column = "event.code"
df_ad_auth_time_column = "_time"
df_ad_auth_sourceip = "SourceIP"

##### Msticpy data visualization

In [7]:
# DataViewer(df_ad_auth)
df_ad_auth.head(5)

,_time,index,host.name,event.code,UserAccount,SourceIP
0,2022-02-19T15:49:39.318+0000,wineventlogs,wkst15.magnumtempus.financial,4624,norma.gene,172.16.21.100
1,2022-02-19T15:49:46.482+0000,wineventlogs,wkst15.magnumtempus.financial,4624,norma.gene,172.16.21.100
2,2022-02-19T15:49:48.447+0000,wineventlogs,files.magnumtempus.financial,4624,norma.gene,172.16.50.144
3,2022-02-19T15:49:48.447+0000,wineventlogs,files.magnumtempus.financial,4624,norma.gene,172.16.50.144
4,2022-02-19T15:49:48.447+0000,wineventlogs,files.magnumtempus.financial,4624,norma.gene,172.16.50.144


In [8]:
df_ad_auth[df_ad_auth["event.code"] == 4624]["UserAccount"].value_counts()[:50]

Administrator        1360
ANONYMOUS LOGON       332
pat.risus             146
norma.gene            115
karen.metuens          87
brent.socium           80
clarie.insigni         59
kama.suppetia          57
reggie.habeo           54
chris.mcquay           53
molly.ferio            48
amanda.nuensis         47
timothy.vanidicus      45
estevan.mcnullen       41
lex.perrin             40
donny.indoles          39
autumn.mi              38
stephen.lamna          35
matt.tristique         32
administrator           9
dale.phasle             1
Name: UserAccount, dtype: int64

In [9]:
df_ad_auth.mp_timeline.plot(
   title="Onprem AD authentication result by User",
   group_by=df_ad_auth_user_column,
   source_columns=[df_ad_auth_eventid_column],
   # group_by=df_ad_auth_eventid_column,
   # source_columns=[df_ad_auth_user_column],
   time_column=df_ad_auth_time_column,
   legend="left",
   height=800
)

Loading BokehJS ...

Column(id='1711', ...)

![Onprem AD authentication result by User - screenshot](./bokeh_plot-auth.png)

In [10]:
df_ad_auth[df_ad_auth["event.code"] == 4625].mp_timeline.plot(
   title="Onprem AD authentication result by User",
   group_by=df_ad_auth_user_column,
   source_columns=[df_ad_auth_eventid_column],
   # group_by=df_ad_auth_eventid_column,
   # source_columns=[df_ad_auth_user_column],
   time_column=df_ad_auth_time_column,
   legend="left",
   height=800
)

Loading BokehJS ...

Column(id='3257', ...)

In [11]:
df_ad_auth[df_ad_auth["event.code"] == 4624].mp_timeline.plot(
   title="Onprem AD authentication result by User",
   group_by=df_ad_auth_user_column,
   source_columns=[df_ad_auth_eventid_column],
   # group_by=df_ad_auth_eventid_column,
   # source_columns=[df_ad_auth_user_column],
   time_column=df_ad_auth_time_column,
   legend="left",
   height=800
)

Loading BokehJS ...

Column(id='4407', ...)

In [12]:
# forcing eventid column as string else getting decimal columns in graph
df_ad_auth[df_ad_auth_eventid_column] = df_ad_auth[df_ad_auth_eventid_column].astype(str)
plot_matrix(data=df_ad_auth, x=df_ad_auth_eventid_column, y=df_ad_auth_user_column, title="Matrix Onprem AD auth eventid vs user")

Loading BokehJS ...

Figure(id='4874', ...)

In [13]:
plot_matrix(
    data=df_ad_auth,
    x=df_ad_auth_sourceip, y=df_ad_auth_user_column,
    title="Matrix Onprem AD auth sourceIP vs user",
    dist_count=True,
    # Inverting the size to show rare interactions
    invert=True
)

Loading BokehJS ...

Figure(id='4975', ...)

In [14]:
# Get all successful logins from the identified attacker source IP
# remember event.code is a string (else, you get no results)
df_ad_auth[(df_ad_auth["event.code"] == "4624") & (df_ad_auth["SourceIP"] == "3.129.164.140")]

,_time,index,host.name,event.code,UserAccount,SourceIP
136,2022-02-19T18:03:28.825+0000,wineventlogs,rdp01.magnumtempus.financial,4624,pat.risus,3.129.164.140
141,2022-02-19T18:03:31.989+0000,wineventlogs,rdp01.magnumtempus.financial,4624,pat.risus,3.129.164.140
142,2022-02-19T18:03:31.989+0000,wineventlogs,rdp01.magnumtempus.financial,4624,pat.risus,3.129.164.140
1087,2022-02-19T20:08:32.358+0000,wineventlogs,rdp01.magnumtempus.financial,4624,brent.socium,3.129.164.140
1096,2022-02-19T20:08:32.359+0000,wineventlogs,rdp01.magnumtempus.financial,4624,clarie.insigni,3.129.164.140
1101,2022-02-19T20:08:33.466+0000,wineventlogs,rdp01.magnumtempus.financial,4624,dale.phasle,3.129.164.140
1139,2022-02-19T20:08:36.660+0000,wineventlogs,rdp01.magnumtempus.financial,4624,norma.gene,3.129.164.140
1156,2022-02-19T20:08:36.661+0000,wineventlogs,rdp01.magnumtempus.financial,4624,pat.risus,3.129.164.140
1198,2022-02-19T20:11:18.753+0000,wineventlogs,rdp01.magnumtempus.financial,4624,pat.risus,3.129.164.140
1200,2022-02-19T20:11:20.762+0000,wineventlogs,rdp01.magnumtempus.financial,4624,pat.risus,3.129.164.140


Just with authentication logs, we know
* Attack vector: password spray
* Source IP of attacker
* Not just one account compromised, but five

We don't know
* Are accounts privileged?
* Any other entry points?
* What activities happen after?

#### OODA 2+
Each observation is a loop that helps refine a plan and actions and get confidence that knowns are more than unknowns.
What attacker has done? escalation of privilege, recon, lateral movement...
* More logs: Password reset, MFA change, Cloud or Saas App token change...
* May want to get VM snapshot or more granular artifacts like process activity, network activity, autoruns...
* Add telemetry if missing (host agent, logs collection, network like IDS/IPS or WAF...)
* Inform Management

Skipping here as overlap with Late alert case

### ... > *Containment* > ...
Business impact should always be reviewed and actions balanced between costs and benefits.
You are unlikely to stop a service that brings millions of revenue if small impact.
But if an attacker is exfiltrating your customers data or your company's intellectual property, it is very likely that action should be taken immediately.
Those questions should be asked to your leadership early on, ideally in Preparation phase.

#### OODA 1
* Lock/Disable affected user accounts

#### OODA 2+
* Contain/Isolate compromised system or suspicious ones if previous action not enough
* Freeze mail server queue, recall or delete emails if email propagation

### ... > *Eradication* > ...

#### OODA 1
* Rebuild compromised asset(s)
* If you can't, cleaning may be an option or not depending on executives choice

#### OODA 2+
* Validate system is functional and protected

### ... > *Recovery* > ...

#### OODA 1
* Return to normal service
* Validate

## Late Alert

Security Alert was raised on company jumphost - Mimikatz execution detection. 
What are you doing?

### ... > *Identification* > ...
A good documentation (inventory, identity matrix, network matrix...) will be of great help but can't replace helpful staff.

#### OODA 1
* No pentest or legitimate activity by IT or known users


#### OODA 2+
Mimikatz execution on RDP jumphost confirmed not legitimate
* Which credentials were stolen? Are they reusable?
  * end-user accounts, service accounts, cloud accounts...
  * domain credentials: privileged, GPO restrictions (remote vs local use, interactive, service...)...
  * local credentials: shared password, use of LAPS (Local Administrator Password Solution = unique local admin password per system)...
* What attacker actions occurred?
* Was there data exfiltration? active or not?


##### Msticpy processtree and recon

In [ ]:
query_win_process4688 = '''
index=wineventlogs event.code=4688 host.name=rdp01
| table _time index event.code winlog.event_data.SubjectUserName winlog.event_data.TargetUserName winlog.event_data.NewProcessName winlog.event_data.ParentProcessName
'''
query_win_process1 = '''
index=sysmon host.name="rdp01.magnumtempus.financial" event.code=1 user.name=pat.risus process.command_line!="*teams*"
| table _time index event.code user.name process.working_directory process.command_line process.parent.command_line
'''
df_win_process = splunk_prov.exec_query(query_win_process1)

In [15]:
df_win_process = pd.read_csv(os.path.join(data_path, 'kc3-sysmon-rdp01-1-pat.risus.csv'))

In [16]:
df_win_process.head(5)

,_time,event.action,event.category{},event.code,event.created,event.provider,host,host.architecture,host.hostname,host.name,...,process.pe.original_file_name,process.pe.product,process.pid,process.thread.id,process.working_directory,rule.name,user.domain,user.id,user.name,winlog.event_data.LogonId
0,2022-02-19T21:27:55.000+0000,Process Create (rule: ProcessCreate),process,1,2022-02-19T21:27:53.917Z,Microsoft-Windows-Sysmon,127.0.0.1:8088,x86_64,rdp01,rdp01.magnumtempus.financial,...,NOTEPAD.EXE,Microsoft® Windows® Operating System,8540,NaN,C:\Users\brent.socium\Desktop\,"technique_id=T1204,technique_name=User Execution",MAGNUMTEMPUS,S-1-5-18,pat.risus,0x6082d1
1,2022-02-19T21:27:28.000+0000,Process Create (rule: ProcessCreate),process,1,2022-02-19T21:27:26.875Z,Microsoft-Windows-Sysmon,127.0.0.1:8088,x86_64,rdp01,rdp01.magnumtempus.financial,...,NOTEPAD.EXE,Microsoft® Windows® Operating System,5692,NaN,C:\Users\brent.socium\,"technique_id=T1204,technique_name=User Execution",MAGNUMTEMPUS,S-1-5-18,pat.risus,0x6082d1
2,2022-02-19T21:18:30.000+0000,Process Create (rule: ProcessCreate),process,1,2022-02-19T21:18:28.938Z,Microsoft-Windows-Sysmon,127.0.0.1:8088,x86_64,rdp01,rdp01.magnumtempus.financial,...,whoami.exe,Microsoft® Windows® Operating System,8320,NaN,C:\Users\brent.socium\Desktop\,"technique_id=T1033,technique_name=System Owner...",MAGNUMTEMPUS,S-1-5-18,pat.risus,0x6060a6
3,2022-02-19T21:18:01.000+0000,Process Create (rule: ProcessCreate),process,1,2022-02-19T21:18:00.682Z,Microsoft-Windows-Sysmon,127.0.0.1:8088,x86_64,rdp01,rdp01.magnumtempus.financial,...,whoami.exe,Microsoft® Windows® Operating System,4712,NaN,C:\Users\brent.socium\Desktop\,"technique_id=T1033,technique_name=System Owner...",MAGNUMTEMPUS,S-1-5-18,pat.risus,0x6060a6
4,2022-02-19T21:17:49.000+0000,Process Create (rule: ProcessCreate),process,1,2022-02-19T21:17:48.479Z,Microsoft-Windows-Sysmon,127.0.0.1:8088,x86_64,rdp01,rdp01.magnumtempus.financial,...,whoami.exe,Microsoft® Windows® Operating System,8864,NaN,C:\Users\brent.socium\Desktop\,"technique_id=T1033,technique_name=System Owner...",MAGNUMTEMPUS,S-1-5-18,pat.risus,0x6060a6


In [17]:
# LX_EVENT_SCH, WIN_EVENT_SCH and MDE_EVENT_SCH are available
## msticpy 2.0/coming
#from msticpy.transform.proc_tree_builder import WIN_EVENT_SCH
## msticpy <2.0
from msticpy.sectools.proc_tree_builder import WIN_EVENT_SCH
from copy import copy
cust_sysmon1_schema = copy(WIN_EVENT_SCH)

cust_sysmon1_schema.time_stamp = "_time"
cust_sysmon1_schema.host_name_column = "host.hostname"
cust_sysmon1_schema.user_name = "user.name"
cust_sysmon1_schema.user_id = "user.id"
cust_sysmon1_schema.process_id = "process.pid"
cust_sysmon1_schema.process_name = "process.name"
cust_sysmon1_schema.parent_id = "process.parent.pid"
cust_sysmon1_schema.parent_name = "process.parent.name"
cust_sysmon1_schema.cmd_line = "process.command_line"
# Note these are used to filter events if you have a data
# set that contains mixed event types.
cust_sysmon1_schema.event_id_column = "event.code"
cust_sysmon1_schema.event_id_identifier = None
cust_sysmon1_schema.logon_id = "winlog.event_data.LogonId"
cust_sysmon1_schema.target_logon_id = None

In [18]:
p_tree_win = ptree.build_process_tree(df_win_process, show_summary=True, schema=cust_sysmon1_schema)

{'Processes': 577, 'RootProcesses': 12, 'LeafProcesses': 302, 'BranchProcesses': 263, 'IsolatedProcesses': 0, 'LargestTreeDepth': 5}


In [19]:
ptree.plot_process_tree(data=df_win_process, legend_col="user.name", show_table=False,
    pid_fmt="dec", schema=cust_sysmon1_schema)

Loading BokehJS ...

(Figure(id='5077', ...), Row(id='5191', ...))

![Process Tree rdp01 pat risus - screenshot](./bokeh_plot-processtree-whoami.png)

In [20]:
df_win_process[df_win_process["process.command_line"].str.contains("csc.exe")][["process.command_line"]].head(5)
# DataViewer(df_win_process[df_win_process["process.command_line"].str.contains("csc.exe")][["process.command_line"]])

,process.command_line
27,"""C:\Windows\Microsoft.NET\Framework64\v4.0.303..."
28,"""C:\Windows\Microsoft.NET\Framework64\v4.0.303..."
29,"""C:\Windows\Microsoft.NET\Framework64\v4.0.303..."
30,"""C:\Windows\Microsoft.NET\Framework64\v4.0.303..."
31,"""C:\Windows\Microsoft.NET\Framework64\v4.0.303..."


In [21]:
df_win_process[df_win_process["process.command_line"].str.contains("csc.exe")][["process.command_line"]].shape

(257, 1)

In [ ]:
# no data
# DataViewer(df_win_process[df_win_process["process.command_line"].str.contains("psexec")][["process.command_line"]])

In [ ]:
# df_win_process[df_win_process["process.command_line"].str.contains("findstr")][["process.command_line"]]
# DataViewer(df_win_process[df_win_process["process.pid"] == 6104][["process.command_line"]])

What we know
* recon activity happened (whoami, tasklist...)
* process memory dump (MiniDump...)
* lateral movement (powershell+csc with computers list resulting from PowerSploit Invoke-Portscan.ps1 here - New-ScriptBlockCallback / AddType)

Missing Mimikatz, psexec: Keep in mind that data can have been tampered (by attacker or inadvertently, wrong filter), maybe software bug, or configuration issue. That's one reason to have multiple data sources and validate they all goes into the same direction.

In [ ]:
# zeek logs
df_zeek = pd.read_csv(os.path.join(data_path, 'kc3-zeek.csv'))

In [ ]:
df_zeek.head(5)

In [ ]:
plot_matrix(
    data=df_zeek,
    x="id.orig_h",
    y="id.resp_h",
    title="External IP flows (intersection)",
    intersect=True,
    sort="asc",
)

In [ ]:
plot_matrix(
    data=df_zeek,    
    x="id.orig_h",
    y="id.resp_h",
    value_col="duration",
    title="External IP flows (rare flows == larger)",
    invert=True,
    sort="asc",
)

* 20.42.*, 52.182.141.63... Ms Windows telemetry noise (see dns resolutions with tools like riskiq

In [ ]:
exclude_src_ip = [
    # Microsoft Windows telemetry
    '20.42.65.85', '20.42.65.89', '20.42.65.90', '20.42.72.131', '20.42.73.26', '52.182.141.63', '52.182.143.210',
    '40.126.29.12'
    # Google
    '142.250.191.163'
    # continue...
    ]
plot_matrix(
    data=df_zeek[~df_zeek["id.resp_h"].isin(exclude_src_ip)],
    x="id.orig_h",
    y="id.resp_h",
    value_col="duration",
    title="External IP flows (rare flows == larger)",
    invert=True,
    sort="asc",
)

In [22]:
df_zeek_dns = pd.read_json(os.path.join(data_path, 'kc3-zeek_s3_dns.log'), lines=True)
# https://umbrella.cisco.com/blog/cisco-umbrella-1-million
umbrella1m = pd.read_csv(os.path.join(data_path, 'top-1m.csv'), names=['count','domain'], header=None)

In [23]:
df_zeek_dns.head(5)

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,trans_id,rtt,query,...,rcode,rcode_name,AA,TC,RD,RA,Z,answers,TTLs,rejected
0,1.645297e+09,CObnBq3KuwlDDVB3Ee,172.16.50.139,61719,172.16.50.100,53,udp,2395,0.012779,officecdn.microsoft.com,...,0.0,NOERROR,False,False,True,True,0,"[2-01-3f20-0003.cdx.cedexis.net, office-prod.e...","[50.0, 50.0, 219.0, 219.0, 20.0]",False
1,1.645297e+09,CDOpU43sSkPdVriVK5,172.16.50.139,57558,172.16.50.100,53,udp,15144,NaN,officecdn.microsoft.com.edgesuite.net,...,0.0,NOERROR,False,False,True,True,0,[officecdn.microsoft.com.edgesuite.net.globalr...,"[19070.0, 1949.0, 20.0, 20.0, 20.0, 20.0]",False
2,1.645297e+09,CpsB2j2YcTqjYn4fYj,172.16.50.136,55701,172.16.50.100,53,udp,3040,0.000251,nexusrules.officeapps.live.com,...,0.0,NOERROR,False,False,True,True,0,"[prod.nexusrules.live.com.akadns.net, 52.109.7...","[2035.0, 144.0]",False
3,1.645297e+09,CSzyL03trQ7YN3n0sd,172.16.50.132,56916,172.16.50.100,53,udp,54687,NaN,_ldap._tcp.dc._msdcs.magnumtempus.financial,...,0.0,NOERROR,True,False,True,True,0,"[dc02.magnumtempus.financial, dc.magnumtempus....","[600.0, 600.0]",False
4,1.645297e+09,C5PsglldQW1sTBQxe,172.16.50.132,53379,172.16.50.100,53,udp,64435,NaN,dc02.magnumtempus.financial,...,0.0,NOERROR,True,False,True,True,0,[172.16.50.101],[3600.0],False


In [24]:
# add tld column
import tldextract
df_zeek_dns['domain'] = df_zeek_dns['query'].apply(lambda x: tldextract.extract(x).domain + '.' + tldextract.extract(x).suffix)

In [25]:
# Can filter out NXDOMAIN and SERVFAIL initially but can still have relevant data
df_zeek_dns[["id.orig_h", "query", "rcode_name"]].groupby(["rcode_name"]).count()

,id.orig_h,query
rcode_name,,
NOERROR,98062,98062
NXDOMAIN,3414,3414
SERVFAIL,1072,1072


In [26]:
# quick look at client dns queries count
df_zeek_dns[["id.orig_h", "query"]].groupby(["id.orig_h"]).count()

,query
id.orig_h,
172.16.50.100,293
172.16.50.101,742
172.16.50.110,896
172.16.50.130,9928
172.16.50.131,9340
172.16.50.132,3153
172.16.50.133,709
172.16.50.134,5565
172.16.50.135,10410


In [27]:
# Top 5 tld queried
df_zeek_dns[["id.orig_h", "domain"]].groupby("domain").count().sort_values(by="id.orig_h", ascending=False).head(5)

,id.orig_h
domain,
magnumtempus.financial,3825
google.com,3621
googleapis.com,3310
amazonaws.com,3307
doubleclick.net,3085


In [28]:
# Aggregate data by count 
df_zeek_dns_agg1 = df_zeek_dns[["id.orig_h", "query", "domain", "answers"]].groupby(["id.orig_h", "query", "domain"]).agg('count').reset_index()

In [29]:
df_zeek_dns_agg1.head(5)

,id.orig_h,query,domain,answers
0,172.16.50.100,1028-ms-7.1695-70399197.ce46c125-8098-11ec-4fa...,ce46c125-8098-11ec-4fab-06fd33b7caf8.,2
1,172.16.50.100,1028-ms-7.1697-70550906.ce46c125-8098-11ec-4fa...,ce46c125-8098-11ec-4fab-06fd33b7caf8.,2
2,172.16.50.100,1028-ms-7.1699-70708085.ce46c125-8098-11ec-4fa...,ce46c125-8098-11ec-4fab-06fd33b7caf8.,2
3,172.16.50.100,1028-ms-7.1701-708bf7e4.ce46c125-8098-11ec-4fa...,ce46c125-8098-11ec-4fab-06fd33b7caf8.,4
4,172.16.50.100,1028-ms-7.1703-70a76f53.ce46c125-8098-11ec-4fa...,ce46c125-8098-11ec-4fab-06fd33b7caf8.,2


In [30]:
df_zeek_dns_agg1.shape

(17381, 4)

In [31]:
# Filter out Umbrella 1 million. this can still contain relevant activities but for initial analysis, helpful to reduce noise
df_zeek_dns_agg1_filtered1 = df_zeek_dns_agg1[
    (~df_zeek_dns_agg1["query"].isin(umbrella1m['domain'].unique().tolist()))
]
# shape give the size of dataframe and validate our progress in data reduction
df_zeek_dns_agg1_filtered1.shape

(1774, 4)

In [32]:
# More contextual filtering based on observed traffic and company
exclude_dst_fqdn = [
    'ec2.us-east-2.amazonaws.com',
    'wpad.us-east-2.ec2-utilities.amazonaws.com',
    'wpad.us-east-2.compute.internal'
    'defcon-2022-obsidian-bq2am.s3.us-east-2.amazonaws.com',
    'pagead2.googlesyndication.com',
    'googleads.g.doubleclick.net',
    'fonts.gstatic.com',
    'adservice.google.com',
    'mcs-va.tiktok.com',
    'content-autofill.googleapis.com',
    'www.google-analytics.com',
    'update.googleapis.com',
    'ping.chartbeat.net',
    'tpc.googlesyndication.com',
    'htlb.casalemedia.com',
    'mobile.pipe.aria.microsoft.com',
    'ib.adnxs.com',
    'fonts.googleapis.com',
    'mon-va.byteoversea.com',
    'stats.g.doubleclick.net',
    'redirect.prod.experiment.routing.cloudfront.aws.a2z.com'
    ]
exclude_dst_domain = [
   'magnumtempus.financial',
   'google.com',
   'googleapis.com',
   'amazonaws.com',
   'doubleclick.net',
   'microsoft.com',
   'gstatic.com',
   'googlesyndication.com',
   'yahoo.com',
   'internal.',
   'tiktok.com',
   'casalemedia.com',
   'rubiconproject.com',
   'pubmatic.com',
   'ip-172-16-55-120.',
   'office.net',
   'googlevideo.com',
   'tiktokcdn.com',
   'thunderbird.net',
   'amazon-adsystem.com',
   'adsafeprotected.com',
   'magnumtempusfinancial.com',
   'adnxs.com',
   'addthis.com',
   'clarity.ms',
   'moatads.com',
   'doubleverify.com',
   'taboola.com',
   'cnn.com',
   'media.net',
   'rlcdn.com',
   'tiktokcdn-us.com',
   'bidswitch.net',
   'cloudfront.net',
   'youtube.com',
   'outbrain.com',
   'indeed.com',
   'google-analytics.com',
   'openx.net',
   'dotomi.com',
   'facebook.com',
   'smartadserver.com',
   'googleusercontent.com',
   'chartbeat.net',
   'demdex.net',
   'krxd.net',
   'googleadservices.com',
   'live.com',
   '2mdn.net',
   'advertising.com',
   'awswaf.com',
    'ubuntu.com',
    'osquery.io',
    'elastic.co',
    # chrome?
    'ce46c125-8098-11ec-4fab-06fd33b7caf8.',
    'eb50ee7a-8720-11ec-4fab-06fd33b7caf8.',
    'c7ec359a-871e-11ec-3cab-06bc168f2a36.',
    'e1a36a4d-8722-11ec-3cab-06bc168f2a36.',
    '172.in-addr.arpa',
    'akamaihd.net',
    'imrworldwide.com',
    'b2c.com',
    'yahoodns.net',
    'online-metrix.net',
    'office.com',
    'cloudmaestro.com',
    'appsflyer.com',
    'wal.co'
   ]
df_zeek_dns_agg1_filtered2 = df_zeek_dns_agg1_filtered1[
    (~df_zeek_dns_agg1_filtered1["query"].isin(exclude_dst_fqdn)) & 
    (~df_zeek_dns_agg1_filtered1["domain"].isin(exclude_dst_domain))
]
df_zeek_dns_agg1_filtered2.shape

(444, 4)

In [33]:
df_zeek_dns_agg1_filtered2.head(5)

,id.orig_h,query,domain,answers
162,172.16.50.110,client.wns.windows.com,windows.com,6
185,172.16.50.110,mangnumtempusfinacial.com,mangnumtempusfinacial.com,0
396,172.16.50.130,aufp.io,aufp.io,2
401,172.16.50.130,autodiscover.magnumtempusfinancial.onmicrosoft...,onmicrosoft.com,5
431,172.16.50.130,blogs.wrldbank.org,wrldbank.org,0


In [34]:
# External DNS flow from source IP addresses to tld
plot_matrix(
    data=df_zeek_dns_agg1_filtered2,
    x="id.orig_h",
    y="domain",
    value_col="answers",
    title="External DNS flows",
    sort="asc",
)

Loading BokehJS ...

Figure(id='5302', ...)

In [35]:
df_zeek_dns_agg1_filtered2[df_zeek_dns_agg1_filtered2['id.orig_h'] == '172.16.55.110'].shape

(9, 4)

In [36]:
# From Zeek DNS logs after filtering, review rdp jumphost
# as identified previously as compromised system
df_zeek_dns_agg1_filtered2[df_zeek_dns_agg1_filtered2['id.orig_h'] == '172.16.55.110']

,id.orig_h,query,domain,answers
17179,172.16.55.110,app.interactsh.com,interactsh.com,4
17182,172.16.55.110,c88nc6r2vtc00001pg0ggrrksdcyyyyyb.interact.sh,interact.sh,4
17186,172.16.55.110,client.wns.windows.com,windows.com,12
17243,172.16.55.110,file.pizza,file.pizza,4
17244,172.16.55.110,files.magnumtempus.finacial,finacial.,0
17246,172.16.55.110,files.magnumtempusfinacial.com,magnumtempusfinacial.com,0
17282,172.16.55.110,relay.webwormhole.io,webwormhole.io,4
17294,172.16.55.110,transfer.sh,transfer.sh,4
17301,172.16.55.110,webwormhole.io,webwormhole.io,4


What we know
* Likely attacker exfiltration: interactsh.com, file.pizza, webwormhole.io, transfer.sh unless known legitimate business use case
* Some typo requests on company domain suggesting manual execution

*Review Forensics station walk-through for more*

In [37]:
# Zeek certificate logs
df_zeek_x509 = pd.read_json(os.path.join(data_path, 'kc3-zeek_s3_x509.log'), lines=True)

In [38]:
df_zeek_x509.shape

(3034, 16)

In [39]:
df_zeek_x509.head(5)

,ts,id,certificate.version,certificate.serial,certificate.subject,certificate.issuer,certificate.not_valid_before,certificate.not_valid_after,certificate.key_alg,certificate.sig_alg,certificate.key_type,certificate.key_length,certificate.exponent,san.dns,basic_constraints.ca,certificate.curve
0,1.645297e+09,FZblKX3kaIGrnbDduh,3,5DD5B12BC46A9D868E8EBE18DA7FD45B30E8BBCE,"emailAddress=admin@magnumtempus.financial,CN=l...","emailAddress=admin@magnumtempus.financial,CN=l...",1639786671,1955146671,rsaEncryption,sha256WithRSAEncryption,rsa,4096,65537.0,[logstash.magnumtempus.financial],NaN,NaN
1,1.645297e+09,FBsMiESgMoy88n6D9,3,0FCBCCB22D39833573D6D40E52A1633F,CN=ec2.us-east-2.amazonaws.com,"CN=Amazon,OU=Server CA 1B,O=Amazon,C=US",1641772800,1673308799,rsaEncryption,sha256WithRSAEncryption,rsa,2048,65537.0,"[ec2.us-east-2.amazonaws.com, us-east-2.ec2.am...",0.0,NaN
2,1.645297e+09,Fhjfaz4x8Eey4Yvo15,3,0FCBCCB22D39833573D6D40E52A1633F,CN=ec2.us-east-2.amazonaws.com,"CN=Amazon,OU=Server CA 1B,O=Amazon,C=US",1641772800,1673308799,rsaEncryption,sha256WithRSAEncryption,rsa,2048,65537.0,"[ec2.us-east-2.amazonaws.com, us-east-2.ec2.am...",0.0,NaN
3,1.645297e+09,F4YRmg4UIrsGU1z6N9,3,0FCBCCB22D39833573D6D40E52A1633F,CN=ec2.us-east-2.amazonaws.com,"CN=Amazon,OU=Server CA 1B,O=Amazon,C=US",1641772800,1673308799,rsaEncryption,sha256WithRSAEncryption,rsa,2048,65537.0,"[ec2.us-east-2.amazonaws.com, us-east-2.ec2.am...",0.0,NaN
4,1.645297e+09,FrAf6j1XNmqpsHKKl5,3,5DD5B12BC46A9D868E8EBE18DA7FD45B30E8BBCE,"emailAddress=admin@magnumtempus.financial,CN=l...","emailAddress=admin@magnumtempus.financial,CN=l...",1639786671,1955146671,rsaEncryption,sha256WithRSAEncryption,rsa,4096,65537.0,[logstash.magnumtempus.financial],NaN,NaN


In [40]:
df_zeek_x509["san.dns"].value_counts()[:10]

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[ec2.us-east-2.amazonaws.com, us-east-2.ec2.amazonaws.com, *.ec2.us-east-2.vpce.amazonaws.com, api.ec2.us-east-2.aws, ec2.us-east-2.api.aws]                                                                                                                                                                                                                                                                                                                                                    1492
[logstash.magnumtempus.financial]                                                                                                                                                                                                                                                                                                                                                                                                                                                                493
[*.s3.us-east-2.amazonaws.com,

In [41]:
df_zeek_x509["san.dns"].value_counts()[-10:]

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[www.bing.com, dict.bing.com.cn, *.platform.bing.com, *.bing.com, bing.com, ieonline.microsoft.com, *.windowssearch.com, cn.ieonline.microsoft.com, *.origin.bing.com, *.mm.bing.net, *.api.bing.com, ecn.dev.virtualearth.net, *.cn.bing.net, *.cn.bing.com, ssl-api.bing.com, ssl-api.bing.net, *.api.bing.net, *.bingapis.com, bingsandbox.com, feedback.microsoft.com, insertmedia.bing.office.net, r.bat.bing.com, *.r.bat.bing.com, *.dict.bing.com.cn, *.dict.bing.com, *.ssl.bing.com, *.appex.bing.com, *.platform.cn.bing.com, wp.m.bing.com, *.m.bing.com, global.bing.com, windowssearch.com, search.msn.com, *.bingsandbox.com, *.api.tiles.ditu.live.com, *.ditu.live.com, *.t0.tiles.ditu.live.com, *.t1.tiles.ditu.live.com, *.t2.tiles.ditu.live.com, *.t3.tiles.ditu.live.com, *.tiles.ditu.live.com, 3d.live.com, api.search.live.com, beta.search.live.com, cnweb.search.live.com, dev.live.com, ditu.live.com, farecast.live.com, image.live.com, images.live.com, local.live.com.au, localsearch.live.com, ls4d.se

In [42]:
df_zeek_x509["certificate.issuer"].value_counts()[:50]

CN=Amazon,OU=Server CA 1B,O=Amazon,C=US                                                                       1910
emailAddress=admin@magnumtempus.financial,CN=logstash.magnumtempus.financial,O=magnumtempus.financial,C=US     493
CN=rdp01.magnumtempus.financial                                                                                216
CN=Microsoft RSA TLS CA 02,O=Microsoft Corporation,C=US                                                        112
CN=Microsoft Azure TLS Issuing CA 02,O=Microsoft Corporation,C=US                                               88
CN=Microsoft Azure TLS Issuing CA 06,O=Microsoft Corporation,C=US                                               29
CN=Microsoft RSA TLS CA 01,O=Microsoft Corporation,C=US                                                         23
CN=DigiCert SHA2 Secure Server CA,O=DigiCert Inc,C=US                                                           22
CN=GTS CA 1C3,O=Google Trust Services LLC,C=US                                  

In [57]:
certificate_issuer_flopX = df_zeek_x509["certificate.issuer"].value_counts()[-10:].index.tolist()

In [58]:
df_zeek_x509[df_zeek_x509["certificate.issuer"].isin(certificate_issuer_flopX)][['certificate.subject', 'certificate.issuer', 'san.dns']]

,certificate.subject,certificate.issuer,san.dns
56,"CN=settings-win.data.microsoft.com,OU=WSE,O=Mi...","CN=Microsoft Secure Server CA 2011,O=Microsoft...",[settings-win.data.microsoft.com]
97,"CN=settings-win.data.microsoft.com,OU=WSE,O=Mi...","CN=Microsoft Secure Server CA 2011,O=Microsoft...",[settings-win.data.microsoft.com]
140,"CN=api.snapcraft.io,O=Canonical Group Ltd,L=Lo...","CN=DigiCert TLS RSA SHA256 2020 CA1,O=DigiCert...","[api.snapcraft.io, api.charmhub.io, assertions..."
345,CN=EC2AMAZ-5JTRQ69,CN=EC2AMAZ-5JTRQ69,"[EC2AMAZ-5JTRQ69, EC2AMAZ-5JTRQ69]"
346,"CN=config.office.com,O=Microsoft Corporation,L...","CN=DigiCert Cloud Services CA-1,O=DigiCert Inc...","[config.office.com, canary.config.office.com, ..."
354,CN=EC2AMAZ-5JTRQ69,CN=EC2AMAZ-5JTRQ69,"[EC2AMAZ-5JTRQ69, EC2AMAZ-5JTRQ69]"
378,"CN=www.github.com,O=GitHub\, Inc.,L=San Franci...","CN=DigiCert SHA2 High Assurance Server CA,OU=w...","[www.github.com, *.github.com, github.com, *.g..."
402,"CN=*.events.data.microsoft.com,O=Microsoft Cor...","CN=Microsoft Azure TLS Issuing CA 01,O=Microso...","[*.events.data.microsoft.com, events.data.micr..."
403,"CN=*.events.data.microsoft.com,O=Microsoft Cor...","CN=Microsoft Azure TLS Issuing CA 01,O=Microso...","[*.events.data.microsoft.com, events.data.micr..."
457,"CN=www.github.com,O=GitHub\, Inc.,L=San Franci...","CN=DigiCert SHA2 High Assurance Server CA,OU=w...","[www.github.com, *.github.com, github.com, *.g..."


In [59]:
certificate_subject_flopX = df_zeek_x509["certificate.subject"].value_counts()[-10:].index.tolist()

In [60]:
df_zeek_x509[df_zeek_x509["certificate.subject"].isin(certificate_subject_flopX)][['certificate.subject', 'certificate.issuer', 'san.dns']]

,certificate.subject,certificate.issuer,san.dns
35,CN=*.google.com,"CN=GTS CA 1C3,O=Google Trust Services LLC,C=US","[*.google.com, *.appengine.google.com, *.bdn.d..."
140,"CN=api.snapcraft.io,O=Canonical Group Ltd,L=Lo...","CN=DigiCert TLS RSA SHA256 2020 CA1,O=DigiCert...","[api.snapcraft.io, api.charmhub.io, assertions..."
346,"CN=config.office.com,O=Microsoft Corporation,L...","CN=DigiCert Cloud Services CA-1,O=DigiCert Inc...","[config.office.com, canary.config.office.com, ..."
513,"CN=*.sysinternals.com,O=Microsoft Corporation,...","CN=Microsoft Azure TLS Issuing CA 02,O=Microso...","[sysazlive.centralus.cloudapp.azure.com, sysaz..."
868,CN=dc02.magnumtempus.financial,CN=dc02.magnumtempus.financial,NaN
1074,CN=*.google.com,"CN=GTS CA 1C3,O=Google Trust Services LLC,C=US","[*.google.com, *.appengine.google.com, *.bdn.d..."
1179,"CN=api.snapcraft.io,O=Canonical Group Ltd,L=Lo...","CN=DigiCert TLS RSA SHA256 2020 CA1,O=DigiCert...","[api.snapcraft.io, api.charmhub.io, assertions..."
1385,"CN=config.office.com,O=Microsoft Corporation,L...","CN=DigiCert Cloud Services CA-1,O=DigiCert Inc...","[config.office.com, canary.config.office.com, ..."
1552,"CN=*.sysinternals.com,O=Microsoft Corporation,...","CN=Microsoft Azure TLS Issuing CA 02,O=Microso...","[sysazlive.centralus.cloudapp.azure.com, sysaz..."
1907,CN=dc02.magnumtempus.financial,CN=dc02.magnumtempus.financial,NaN


In [68]:
# anomaly: transfer.sh is using letsencrypt. gap in visibility?
df_zeek_x509[df_zeek_x509["certificate.subject"].str.contains('transfer')]

,ts,id,certificate.version,certificate.serial,certificate.subject,certificate.issuer,certificate.not_valid_before,certificate.not_valid_after,certificate.key_alg,certificate.sig_alg,certificate.key_type,certificate.key_length,certificate.exponent,san.dns,basic_constraints.ca,certificate.curve


In [66]:
# anomaly: webwormhole.io is using letsencrypt. gap in visibility?
df_zeek_x509[df_zeek_x509["certificate.subject"].str.contains('webworm')]

,ts,id,certificate.version,certificate.serial,certificate.subject,certificate.issuer,certificate.not_valid_before,certificate.not_valid_after,certificate.key_alg,certificate.sig_alg,certificate.key_type,certificate.key_length,certificate.exponent,san.dns,basic_constraints.ca,certificate.curve


### ... > *Containment* > ...

Depending on the pace of your investigation, you may stop attacker
* Before or during data exfiltration
* Before data destruction and logs wiping

#### OODA 1
* Lock/Disable affected user accounts
* Restrict network access (by domain, IP address...) to block C2 or prevent exfiltration

#### OODA 2+
New observation: ongoing event logs clearing on many servers including domain controllers

* Escalate to Crisis team. This will usually include someone from each executive branch (Legal, Communication, HR...)
* Inform Management
* Unplug from network or shutdown impacted systems. First one usually prefered as it will help preserve evidence.
* Shutdown network
* Switch to out-of-band communication
* Involve external partners: Insurance, External retainer, Law Enforcement (LE), Information Sharing and Analysis Center (ISAC), National or sector CERT...
* Validate backups - this is an early recovery step but it can influence containment meaning results should be known early
* Pay Ransom - Trolley Problem or "Cornelian dilemma". Think carefully and discuss with relevant parties (hint: the ones above)
* Monitor Internet: traditional media, social media, forums, dark web...

### ... > *Eradication* > ...

#### OODA 1
* Establish priority for services recovery (Business systems, Payroll, AD...)
* Rebuild asset(s)

#### OODA 2+
* Validate systems are functional and protected

### ... > *Recovery* > ...

#### OODA 1
* Return to normal service
* Validate

## Lessons Learned
Final communication with closure followed by incident report

Should include:
* Executive summary
* Key findings, recommendations
* What went good
* What can be improved
* High-level Timeline

Depending on organization, alignment with MITRE Att&CK, MITRE Shield, NIST SP800-53 or ISO2700x may be good additions.
Part or all recommendations should feed your org risk register and be visible to decision-makers.

*More in Final Reporting walk-through*

## Conclusion

* Incident response is usually a marathon.
* Preparation is key! processes, training and teams collaboration.
* You want to assess situation and adapt often. Better and faster than attacker. OODA loop is one method to structure this.
* Jupyter notebook is one tool that can help structure investigation and automate part of it. It helps to show the reasoning of investigation and avoid bias.

## That's all the folks!
Thanks to
* [BTV project obsidian](https://www.blueteamvillage.org/programs/project-obsidian/) contributors
* Projects jupyter notebook, pandas, msticpy, OTRF / [Infosec Jupyterthon](https://infosecjupyterthon.com)

<h1 align="center" style="font-weight: bold;">Thank you</h1>

<div align="center">
Join the conversation<br />
https://discord.blueteamvillage.org
</div>

## References

Links msticpy
* https://github.com/microsoft/msticpy/
* https://msticpy.readthedocs.io/en/latest/GettingStarted.html
* https://msticpy.readthedocs.io/en/latest/visualization/ProcessTree.html

Links OODA Loop
* https://en.wikipedia.org/wiki/OODA_loop
* [Incident Response: Taking CSIRT modeling to the next level, Frode Hommedal](http://frodehommedal.no/presentations/first-tc-oslo-2015/#/slide-start) - [CSIRT OODA Loop](https://www.frodehommedal.no/presentations/first-tc-oslo-2015/#/slide-csirt-ooda-loop)

Links Ransomware
* [Ransomware Guide, CISA, Sep 2020](https://www.cisa.gov/publication/ransomware-guide)
* [CISA’s CSET Tool Sets Sights on Ransomware Threat, Jun 2021](https://us-cert.cisa.gov/ncas/current-activity/2021/06/30/cisas-cset-tool-sets-sights-ransomware-threat)
* [Stop Ransomware, CISA, Jul 2021](https://www.cisa.gov/stopransomware)
* [ENISA Threat Landscape 2020 - Ransomware, Oct 2020](https://www.enisa.europa.eu/publications/ransomware)
* [RANSOMWARE GUIDE, MS-ISAC, SEPTEMBER 2020](https://www.cisa.gov/sites/default/files/publications/CISA_MS-ISAC_Ransomware Guide_S508C_.pdf)
* [Attaques par rançongiciels, tous concernés - Comment les anticiper et réagir en cas d’incident ? ANSSI, Sep 2020](https://www.ssi.gouv.fr/guide/attaques-par-rancongiciels-tous-concernes-comment-les-anticiper-et-reagir-en-cas-dincident/)
* [Ransomware Protection and Containment Strategies, FireEye](https://www.fireeye.com/content/dam/fireeye-www/current-threats/pdfs/wp-ransomware-protection-and-containment-strategies.pdf)
* [Microsoft Enterprise access model, Jun 2021](https://docs.microsoft.com/en-us/security/compass/privileged-access-access-model), previously [Active Directory administrative tier model, Feb 2019](https://docs.microsoft.com/en-us/windows-server/identity/securing-privileged-access/securing-privileged-access-reference-material)
* [Ransomware: How to Prevent and Recover (ITSAP.00.099)](https://cyber.gc.ca/en/guidance/ransomware-how-prevent-and-recover-itsap00099), [Ransomware](https://cyber.gc.ca/en/ransomware)
* [NIST Ransomware Risk Management: A Cybersecurity Framework Profile - 8374](https://www.nist.gov/publications/ransomware-risk-management-cybersecurity-framework-profile)
* [Microsoft DART ransomware approach and best practices](https://docs.microsoft.com/en-us/security/compass/incident-response-playbook-dart-ransomware-approach)

* [Lessons from TV5Monde 2015 Hack, Jun 2017](https://www.comae.com/posts/lessons-from-tv5monde-2015-hack/), [Conférence de clôture: Retour technique de l'incident de TV5Monde  ANSSI - SSTIC - French, Jun 2017](https://www.sstic.org/2017/presentation/2017_cloture/)
* [A Conversation with a Hacker - CWT - operator negotiation, Sep 2020](https://www.triella.com/a-conversation-with-a-hacker/)
* [Ransomware Victims That Pay Up Could Incur Steep Fines from Uncle Sam, Oct 2020](https://krebsonsecurity.com/2020/10/ransomware-victims-that-pay-up-could-incur-steep-fines-from-uncle-sam/), [Advisory on Potential Sanctions Risks for Facilitating Ransomware Payments - OFAC, Oct 2020](https://home.treasury.gov/system/files/126/ofac_ransomware_advisory_10012020_1.pdf)
* [Hospitals take action to avoid ransomware attacks, including pre-emptive email shut down, Nov 2020](https://www.beckershospitalreview.com/cybersecurity/hospitals-take-action-to-avoid-ransomware-attacks-including-pre-emptive-email-shut-down.html)
* [Ransomware: Remove Response Paralysis with a Comprehensive Incident Response Plan](https://www.marsh.com/sg/insights/research/ransomware-removing-response-paralysis.html)
* [Human operated ransomware, Feb 2021](https://docs.microsoft.com/en-us/security/compass/human-operated-ransomware)
* [I've worked a lot of #ransomware incidents and I've found that most companies don't realize what the true cost of a ransomware incident is.But isn't it just paying the ransom or restoring and you're done? Nope. Here are the (potential) costs (based on my experience): (1/X), May 2021](https://twitter.com/SecShoggoth/status/1389645581325320204)
* [BloodHound versus Ransomware: A Defender’s Guide, Jun 2021](https://posts.specterops.io/bloodhound-versus-ransomware-a-defenders-guide-28147dedb73b)
* [Ransomware prevention: How organizations can fight back , Feb 2022](https://www.mckinsey.com/business-functions/risk-and-resilience/our-insights/ransomware-prevention-how-organizations-can-fight-back)

## Known issues, troubleshooting

### Jupyter notebook recommendations

* Notebooks are json files which make them well adapted to store in source control tool like git.
* Msticpy DataViewer is great to review data but if you care about the size of notebook file, it's better to keep only filtered data or use head/tail as preview. Note that graph rendering will include all the input data, no matter what, so better to only input data that is needed.

### Jupyter notebook slideshow
Tested on Jupyterlab 3.2.9

* Menu View > Cell Toolbar > Slideshow = NOK
* Select Cell Tools on the left side bar > Slide Type = NOK
* Left or right bar: Property inspector > Slide Type = OK
* Convert to slides (this seems the only export for which slide type skip applies)
  * Menu File: Export notebook to Reveal.js slides. If rendering as single-page, ensure you have network access to unpkg.com and cdnjs.cloudflare.com
  * `jupyter nbconvert notebook.ipynb --to slides --post serve`
* Convert to pdf
  * Menu File: Export notebook to PDF (requires pandoc texlive)
* No presentation icon in toolbar near save, add cell, cut, paste...
* Color rendering
  * reveal.js theme
  * custom.css
  * html raw edit (references seems to change depending on jupyter version - different with 3.4.3)
    * `/* Headings */` color: `color: white;`
    * body text color: `--jp-content-font-color1: white;`
    * body background color: `--jp-layout-color0: #595959;`

links
* https://www.markroepke.me/posts/2019/06/05/tips-for-slideshows-in-jupyter.htmlhttps://www.markroepke.me/posts/2019/06/05/tips-for-slideshows-in-jupyter.html
* https://towardsdatascience.com/5-slides-for-tips-on-presentation-mode-in-jupyter-notebook-f858b15fff4f
* https://github.com/jupyterlab/jupyterlab/issues/5018https://github.com/jupyterlab/jupyterlab/issues/5018

### Video recording

* [How to record the screen on your Mac](https://support.apple.com/en-ca/HT208721)
* [List of Open Source Screen Recorders for Linux](https://linuxhint.com/list-free-screen-recorders-linux/)
* [Open Broadcaster Software | OBS](https://obsproject.com/)
* [Jitsi Meet, Record, Upload to Dropbox](https://meet.jit.si/)